# Interaktivní vizualizace interakcí RTG záření

Tento notebook interaktivně znázorňuje, jak se mění poměr mezi **fotoelektrickým jevem** a **Comptonovým rozptylem** v závislosti na dvou klíčových faktorech: **energii fotonu (E)** a **protonovém čísle materiálu (Z)**.

## Fyzikální principy ⚛️

Skript využívá zjednodušené, ale pro radiologickou fyziku relevantní vztahy:

* **Pravděpodobnost fotoelektrického jevu** je přibližně úměrná $Z^3 / E^3$. Tento jev je zodpovědný za **kontrast obrazu**. Dominantní je u materiálů s vysokým Z a při nízkých energiích.
* **Pravděpodobnost Comptonova rozptylu** je téměř nezávislá na protonovém čísle (Z) a jen mírně klesá s energií. Tento jev přispívá k **rozptýlenému záření**, které snižuje kvalitu obrazu a zvyšuje dávku pacienta.

Graf ukazuje poměr pravděpodobnosti **Fotoelektrický jev / Comptonův rozptyl**.

- **Hodnota > 1**: Převažuje fotoelektrický jev (zobrazeno červeně).
- **Hodnota < 1**: Převažuje Comptonův rozptyl (zobrazeno modře).

## Jak používat tento notebook?

1.  **Spusťte buňku s kódem:** Klikněte do následující buňky a spusťte ji (např. pomocí klávesové zkratky `Shift + Enter`).
2.  **Vyberte materiál:** Z rozbalovacího menu si zvolte jednu z předdefinovaných tkání nebo kontrastních látek.
3.  **Nastavte vlastní Z:** Chcete-li experimentovat, vyberte v menu "Vlastní" a nastavte protonové číslo pomocí posuvníku.

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import Dropdown, RadioButtons, IntSlider
from IPython.display import display

# Efektivní protonová čísla (orientační – didaktický model)
materialy = {
    'Tuk': 6.3,
    'Voda': 7.4,
    'Svalovina': 7.6,
    'Kost (kompaktní)': 13.8,
    'Vzduch': 7.6,
    'Jod (kontrast)': 53.0,
    'Baryum (kontrast)': 56.0
}

def plot_interaction_ratio(material_name, compton_model, energy_keV):
    # volba Z 
    Z = float(materialy[material_name])
    title_material = f'materiál: {material_name} (Z≈{Z:.1f})'

    # rozsah energií pro vykreslení křivky (fixní); ovládá se pouze E0
    E = np.linspace(10, 150, 600)
    E0 = float(energy_keV)

    # Fotoefekt ~ Z^3 / E^3 (zjednodušeně)
    prob_photo = (Z**3) / (E**3)

    # Jednoduché modely Comptonu
    if compton_model == 'const':
        prob_compton = np.ones_like(E)               # ~E^0
        prob_compton_E0 = 1.0
    else:
        prob_compton = 1.0 / E                       # ~E^-1
        prob_compton_E0 = 1.0 / E0

    ratio = prob_photo / prob_compton               # křivka
    ratio_E0 = (Z**3) / (E0**3) / prob_compton_E0   # hodnota v E0

    # vykreslení
    fig, ax = plt.subplots(figsize=(11, 6.5))

    ax.plot(E, ratio, linewidth=2.5,
            label=f'Poměr P(FE)/P(Compton) pro Z={Z:.1f}')

    # zbarvení oblastí
    ax.fill_between(E, np.minimum(ratio, 1), 1, where=ratio<1, alpha=0.25,
                    label='Převažuje Comptonův rozptyl')
    ax.fill_between(E, 1, np.maximum(ratio, 1), where=ratio>=1, alpha=0.25,
                    label='Převažuje fotoelektrický jev')

    # Poměr = 1
    ax.axhline(1, linestyle='--', linewidth=1.3, label='Poměr = 1')

    # Svislá značka v zadané energii (konstantní energie)
    ax.axvline(E0, linestyle=':', linewidth=1.8)
    ax.plot([E0], [ratio_E0], marker='o', markersize=7)
    ax.text(E0, ratio_E0, f"  E = {E0:.0f} keV,\n  poměr ≈ {ratio_E0:.3g}",
            va='bottom', ha='left')

    ax.set_title(f'Závislost poměru interakcí na energii pro {title_material}', fontsize=14)
    ax.set_xlabel('Energie fotonu [keV]')
    ax.set_ylabel('Poměr pravděpodobností\nFotoelektrický jev / Comptonův rozptyl')
    ax.set_yscale('log')
    ax.grid(True, which='both', ls='--', alpha=0.5)
    ax.set_xlim(10, 150)
    ax.set_ylim(1e-3, 1e3)
    ax.legend()
    plt.show()

# --- OVLÁDACÍ PRVKY ---
material_dropdown = Dropdown(
    options=list(materialy.keys()),
    value='Kost (kompaktní)',
    description='Vyber materiál:',
    style={'description_width': 'initial'},
    layout={'width': '320px'}
)

compton_model = RadioButtons(
    options=[('Konstantní (~E⁰)', 'const'), ('Mírný pokles (~1/E)', 'invE')],
    value='const',
    description='Model Comptonu:',
    style={'description_width': 'initial'}
)

energy_slider = IntSlider(
    value=70, min=10, max=150, step=1,
    description='Energie [keV]:',
    style={'description_width': 'initial'},
    layout={'width': '320px'}
)

# Propojení (bez vlastního Z a bez K-hrany)
out = widgets.interactive_output(
    plot_interaction_ratio,
    {
        'material_name': material_dropdown,
        'compton_model': compton_model,
        'energy_keV': energy_slider
    }
)

ui = widgets.VBox([
    widgets.HBox([material_dropdown, energy_slider]),
    compton_model
])

display(ui, out)


Output()

## Klíčová zjištění z grafu 

* **Vliv energie (E):** Pro jakýkoliv materiál platí, že s **rostoucí energií** fotonů prudce klesá pravděpodobnost fotoelektrického jevu. Proto se poměr rychle posouvá ve prospěch Comptonova rozptylu. To je důvod, proč se pro mamografii (kde je potřeba vysoký kontrast měkkých tkání) používají nízké energie (cca 20-30 keV).

* **Vliv protonového čísla (Z):** Porovnejte například "Svalovinu" (Z≈7.6) a "Kost" (Z≈13.8). Uvidíte, že u kosti je křivka mnohem výše. To znamená, že fotoelektrický jev v kosti dominuje až do mnohem vyšších energií než ve svalovině. Právě tento obrovský rozdíl, daný závislostí na **$Z^3$**, je klíčem k vysokému kontrastu mezi kostí a měkkou tkání na rentgenovém snímku.

* **Kontrastní látky:** Zvolte "Jod" (Z=53) nebo "Baryum" (Z=56). Křivka se posune dramaticky nahoru. Tyto látky masivně absorbují rentgenové záření prostřednictvím fotoelektrického jevu v celém diagnostickém rozsahu energií, a proto se používají ke zviditelnění cév nebo trávicího traktu.